In [29]:
import pandas as pd
import numpy as np
import math
import json

In [2]:
df = pd.DataFrame({
    'Temperature': ['Mild', 'Hot', 'Hot', 'Cool', 'Cool', 'Mild', 'Cool', 'Mild', 'Mild', 'Mild', 'Hot', 'Mild', 'Cool', 'Mild'],
    'Outloook': ['Sunny', 'Sunny', 'Overcast', 'Rain', 'Overcast', 'Sunny', 'Sunny', 'Rain', 'Sunny', 'Overcast', 'Overcast', 'Rain', 'Rain', 'Rain'],
    'Humidity': [80, 75, 77, 70, 72, 77, 70, 69, 65, 77, 74, 77, 73, 78],
    'Windy': ['No', 'Yes', 'No', 'No', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No'],
    'Played': ['Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes']
})

df['Humid'] = df['Humidity'] > 75
df = df.drop('Humidity', axis=1)

df

,Temperature,Outloook,Windy,Played,Humid
0,Mild,Sunny,No,Yes,True
1,Hot,Sunny,Yes,No,False
2,Hot,Overcast,No,Yes,True
3,Cool,Rain,No,Yes,False
4,Cool,Overcast,Yes,Yes,False
5,Mild,Sunny,No,No,True
6,Cool,Sunny,No,Yes,False
7,Mild,Rain,No,Yes,False
8,Mild,Sunny,Yes,Yes,False
9,Mild,Overcast,Yes,Yes,True


In [3]:
def entropy(p):
    return -np.sum(p * np.log2(p))

In [4]:
# Get the probability of a unique feature in the dataset 
def P(unique, feature, data):
    count_unique = data[feature].value_counts().get(unique, 0)
    total_count = data[feature].count()
    if total_count == 0:
        return 0
    return count_unique / total_count

In [91]:
def E(target, feature, data):
    if feature is None:
        unique_target_counts = data[target].value_counts()
        total_target_counts = data[target].count()
        e = entropy(unique_target_counts / total_target_counts)
        print(f'E({target}) = {str(e)}')
        return e
    
    # Get the unique features
    unique_features = data[feature].unique()

    # Do computation
    e = 0
    for unique_feature in unique_features:
        subset = data[data[feature] == unique_feature]
        unique_target_counts = subset[target].value_counts()
        total_target_count = subset[target].count()
        e += P(unique_feature, feature, data) * entropy(unique_target_counts / total_target_count)

    print(f'E({target}|{feature}) = {str(e)}')

    return e

In [92]:
def IG(target, feature, data):
    ig = E(target, None, data) - E(target, feature, data)
    print(f'IG({feature}) = {str(ig)}')
    return ig

In [103]:
def getBestFeature(target, data):
    features = data.drop(target, axis = 1)
    highest_IG = -math.inf
    
    for feature in features.columns:
            ig = IG(target, feature, data)
            if ig > highest_IG:
                highest_IG = ig
                best_feature = feature
    
    return best_feature

In [100]:
class Node:
    def __init__(self, feature=None, result=None):
        self.feature = feature
        self.result = result
        self.isLeaf = result is not None
        self.decision_nodes = []
        self.decision_name = []

    def add_decision_node(self, name, node):
        self.decision_nodes.append(node)
        self.decision_name.append(name)

    def to_string(self, layer = 1):
        if self.isLeaf:
            res = f"{' ' * layer * 2}-{self.result}\n"
        else:
            res = f"{' ' * layer * 2}-{self.feature}\n"
            for i, decision in enumerate(self.decision_name):
                res += f"{' ' * layer * 2} =>{decision}\n"
                res += self.decision_nodes[i].to_string(layer + 2)

        return res

    def __str__(self):
        return self.to_string()

In [110]:
def buildTree(target, data, layer = 1, decisions = []):
    print(f"**Layer {str(layer)} after choosing {', '.join(decisions)}**")
    feature_category = getBestFeature(target, data)

    node = Node(feature_category)

    for feature in data[feature_category].unique():
        node.decision_name.append(feature)        
        subset = data[data[feature_category] == feature].drop(feature_category, axis=1)
        if len(subset.columns) == 1:
            res = []
            for value, count in subset[target].value_counts().items():
                res.append(f"{value}={count}")
            node.decision_nodes.append(Node(None, ', '.join(res)))
        else:
            node.decision_nodes.append(buildTree(target, subset, layer + 1, decisions + [feature]))
    return node

In [111]:
print(buildTree("Played", df))

**Layer 1 after choosing **
E(Played) = 0.863120568566631
E(Played|Temperature) = 0.8601317128547441
IG(Temperature) = 0.0029888557118868953
E(Played) = 0.863120568566631
E(Played|Outloook) = 0.6935361388961918
IG(Outloook) = 0.16958442967043919
E(Played) = 0.863120568566631
E(Played|Windy) = 0.7391796818283407
IG(Windy) = 0.12394088673829029
E(Played) = 0.863120568566631
E(Played|Humid) = 0.8571428571428571
IG(Humid) = 0.0059777114237739015
**Layer 2 after choosing Sunny**
E(Played) = 0.9709505944546686
E(Played|Temperature) = 0.5509775004326937
IG(Temperature) = 0.4199730940219749
E(Played) = 0.9709505944546686
E(Played|Windy) = 0.9509775004326937
IG(Windy) = 0.01997309402197489
E(Played) = 0.9709505944546686
E(Played|Humid) = 0.9509775004326937
IG(Humid) = 0.01997309402197489
**Layer 3 after choosing Sunny, Mild**
E(Played) = 0.9182958340544896
E(Played|Windy) = 0.6666666666666666
IG(Windy) = 0.2516291673878229
E(Played) = 0.9182958340544896
E(Played|Humid) = 0.6666666666666666
IG(H

# Please refer to my pdf file for the final tree